In [ ]:
# Convert timestamp and extract year and month
ratings_time = ratings_df.withColumn("timestamp_dt", to_timestamp(from_unixtime(col("timestamp")))) \
    .withColumn("year", year(col("timestamp_dt"))) \
    .withColumn("month", month(col("timestamp_dt")))

# Categorize as Early Year or Late Year
ratings_time = ratings_time.withColumn("time_of_year", 
    when((col("month") >= 1) & (col("month") <= 6), "Early Year")
    .otherwise("Late Year"))

# Show 10 rows
print("\nSample Rows with Year, Month, Time of Year:")
ratings_time.select("userId", "movieId", "rating", "timestamp", "year", "month", "time_of_year").show(10, truncate=False)

# Count ratings in each half of the year
time_grouped = ratings_time.groupBy("time_of_year").agg(count("*").alias("rating_count")).orderBy("time_of_year")
print("\nNumber of Ratings by Time of Year:")
time_grouped.show()

# Save to S3
output_time_path = f"s3a://{s3_bucket}/time_of_year_rating_distribution.csv"
time_grouped.coalesce(1).write.mode("overwrite").csv(output_time_path, header=True)
print(f"\nTime of year distribution saved to: {output_time_path}")